<a href="https://colab.research.google.com/github/spacey-g/Gaia-stellar-analyzer/blob/main/01_initial_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astroquery astropy pandas matplotlib numpy scikit-learn tqdm


In [ ]:
from astroquery.gaia import Gaia
import pandas as pd

# 1. Query single star
def query_star_by_gaia_id(gaia_id):
    query = f"""
        SELECT *
        FROM gaiadr3.gaia_source
        WHERE source_id = {gaia_id}
    """
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    return result.to_pandas()

# 2. Query multiple stars
def query_multiple_gaia_ids(id_list):
    id_list_str = ",".join(map(str, id_list))
    query = f"""
        SELECT *
        FROM gaiadr3.gaia_source
        WHERE source_id IN ({id_list_str})
    """
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    return result.to_pandas()

# 3. Query region
def query_region(ra, dec, radius_arcmin):
    query = f"""
        SELECT *
        FROM gaiadr3.gaia_source
        WHERE CONTAINS(
            POINT('ICRS', ra, dec),
            CIRCLE('ICRS', {ra}, {dec}, {radius_arcmin/60})
        ) = 1
    """
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    return result.to_pandas()

from astroquery.gaia import Gaia
import pandas as pd
def query_by_magnitude(max_g_mag=12):
    query = f"""
        SELECT TOP 5000
            source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax, parallax_error
        FROM gaiadr3.gaia_source
        WHERE phot_g_mean_mag < {max_g_mag}
    """
    job = Gaia.launch_job(query)  # sync mode for Colab
    return job.get_results().to_pandas()






In [ ]:
from astroquery.gaia import Gaia

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.MAIN_GAIA_URL = "https://gea.esac.esa.int/tap-server/tap"   # mirror
Gaia.ROW_LIMIT = -1  # allow full queries


In [ ]:
df_single = query_star_by_gaia_id(5853498713190525696)
df_single


In [ ]:
ids = [5853498713190525696, 5853498713190524800]
df_multi = query_multiple_gaia_ids(ids)
df_multi


In [ ]:
df_region = query_region(ra=56.0, dec=-23.0, radius_arcmin=5)
df_region


In [ ]:
df_mag = query_by_magnitude(8)
df_mag


In [ ]:
df_mag.to_csv("bright_stars.csv", index=False)


In [ ]:
# Clean data for HR diagram
df_clean = df_mag.dropna(subset=["bp_rp", "phot_g_mean_mag"])
df_clean = df_clean[(df_clean["bp_rp"] > -1) & (df_clean["bp_rp"] < 5)]
df_clean = df_clean[(df_clean["phot_g_mean_mag"] > -5) & (df_clean["phot_g_mean_mag"] < 20)]

df_clean.head()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 11))

scatter = plt.scatter(
    df_clean["bp_rp"],
    df_clean["phot_g_mean_mag"],
    c=df_clean["bp_rp"],  # color by temperature proxy
    cmap="plasma_r",      # reverse plasma = blue hot, red cool
    s=6,
    alpha=0.7,
    edgecolors="none"
)

plt.gca().invert_yaxis()

cbar = plt.colorbar(scatter)
cbar.set_label("BP − RP (Color Index)")

plt.xlabel("BP − RP (Color Index)")
plt.ylabel("G Magnitude")
plt.title("Enhanced Gaia HR Diagram")

plt.grid(alpha=0.2)
plt.show()


In [ ]:
plt.figure(figsize=(9, 11))

plt.hist2d(
    df_clean["bp_rp"],
    df_clean["phot_g_mean_mag"],
    bins=200,
    cmap="magma_r"
)

plt.gca().invert_yaxis()

plt.colorbar(label="Density")
plt.xlabel("BP − RP")
plt.ylabel("G Magnitude")
plt.title("Gaia HR Diagram (Density Map)")

plt.show()


In [ ]:
plt.text(0.3, 5, "White Dwarfs", color="white")
plt.text(1.2, 4, "Main Sequence", color="white")
plt.text(1.8, 1, "Red Giants", color="white")


In [ ]:
import numpy as np

df = df_mag.dropna(subset=["bp_rp", "phot_g_mean_mag", "parallax"])

# keep only positive parallaxes
df = df[df["parallax"] > 0]

# compute distance in parsecs
df["distance_pc"] = 1000 / df["parallax"]

# compute absolute magnitude
df["M_G"] = df["phot_g_mean_mag"] - 5 * np.log10(df["distance_pc"] / 10)

# clean color range
df = df[(df["bp_rp"] > -1) & (df["bp_rp"] < 5)]
df = df[(df["M_G"] > -5) & (df["M_G"] < 15)]

df_clean = df
df_clean.head()


In [ ]:
def bp_rp_to_temp(bp_rp):
    # simple polynomial mapping (gives ~3000K to ~10000K)
    return  8500 - 3500 * (bp_rp - 0.5)

df_clean["temperature_K"] = bp_rp_to_temp(df_clean["bp_rp"])


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 12))

scatter = plt.scatter(
    df_clean["bp_rp"],
    df_clean["M_G"],
    c=df_clean["temperature_K"],
    cmap="inferno_r",    # red = cool, blue = hot
    s=6,
    alpha=0.7,
    edgecolors="none"
)

plt.gca().invert_yaxis()

cbar = plt.colorbar(scatter)
cbar.set_label("Temperature (K)")

plt.xlabel("BP − RP (Color Index)")
plt.ylabel("Absolute Magnitude M_G")
plt.title("Gaia HR Diagram (Enhanced: Temperature, Absolute Magnitude)")

plt.grid(alpha=0.2)
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 12))

sns.kdeplot(
    x=df_clean["bp_rp"],
    y=df_clean["M_G"],
    fill=True,
    cmap="magma",
    thresh=0.05,
    levels=50
)

plt.gca().invert_yaxis()
plt.xlabel("BP − RP")
plt.ylabel("Absolute Magnitude (M_G)")
plt.title("Gaia HR Diagram — KDE Density Map")
plt.show()


In [ ]:
plt.figure(figsize=(10, 12))

plt.scatter(
    df_clean["bp_rp"],
    df_clean["M_G"],
    s=4,
    alpha=0.6
)

plt.gca().invert_yaxis()

# labels
plt.text(0.2, 13, "White Dwarfs", color="cyan", fontsize=14)
plt.text(1.0, 7, "Main Sequence", color="yellow", fontsize=16)
plt.text(1.6, 0, "Red Giants", color="orange", fontsize=16)

plt.xlabel("BP − RP")
plt.ylabel("Absolute Magnitude (M_G)")
plt.title("Gaia HR Diagram with Stellar Regions")
plt.show()


In [ ]:
plt.figure(figsize=(10, 12))

scatter = plt.scatter(
    df_clean["bp_rp"],
    df_clean["M_G"],
    c=np.log10(df_clean["distance_pc"]),
    cmap="viridis",
    s=6,
    alpha=0.7
)

plt.gca().invert_yaxis()
plt.colorbar(scatter, label="log10(Distance pc)")
plt.xlabel("BP − RP")
plt.ylabel("Absolute Magnitude M_G")
plt.title("Gaia HR Diagram — Color by Distance")
plt.show()


In [ ]:
# Prepare data for clustering
df_ml = df_mag[["bp_rp", "phot_g_mean_mag", "parallax", "parallax_error"]].copy()

df_ml = df_ml.dropna()

# remove impossible values
df_ml = df_ml[df_ml["parallax"] > 0]
df_ml = df_ml[(df_ml["bp_rp"] > -1) & (df_ml["bp_rp"] < 5)]
df_ml = df_ml[(df_ml["phot_g_mean_mag"] > -5) & (df_ml["phot_g_mean_mag"] < 20)]

df_ml.head()


In [ ]:
df_ml["distance_pc"] = 1000 / df_ml["parallax"]
df_ml["M_G"] = df_ml["phot_g_mean_mag"] - 5 * np.log10(df_ml["distance_pc"] / 10)

features = df_ml[["bp_rp", "M_G"]]


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_pca)

df_ml["cluster_kmeans"] = labels_kmeans


In [ ]:
plt.figure(figsize=(9, 12))

plt.scatter(
    df_ml["bp_rp"],
    df_ml["M_G"],
    c=df_ml["cluster_kmeans"],
    cmap="viridis",
    s=10,
    alpha=0.7
)

plt.gca().invert_yaxis()
plt.xlabel("BP − RP")
plt.ylabel("Absolute Magnitude M_G")
plt.title("Gaia HR Diagram — K-Means Clustering")
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.15, min_samples=30)
labels_dbscan = dbscan.fit_predict(X_pca)

df_ml["cluster_dbscan"] = labels_dbscan


In [ ]:
plt.figure(figsize=(9, 12))

plt.scatter(
    df_ml["bp_rp"],
    df_ml["M_G"],
    c=df_ml["cluster_dbscan"],
    cmap="tab10",
    s=10,
    alpha=0.7
)

plt.gca().invert_yaxis()
plt.xlabel("BP − RP")
plt.ylabel("Absolute Magnitude M_G")
plt.title("Gaia HR Diagram — DBSCAN Clustering")
plt.show()


In [ ]:
!pip install lightkurve astroquery astropy


In [ ]:
from astroquery.gaia import Gaia

def gaia_dr3_to_dr2(dr3_id):
    """
    Convert Gaia DR3 source_id -> Gaia DR2 source_id using the Gaia DR3 DR2 neighbourhood table.
    """
    query = f"""
        SELECT dr2_source_id
        FROM gaiadr3.dr2_neighbourhood
        WHERE dr3_source_id = {dr3_id}
        ORDER BY angular_distance ASC
        LIMIT 1
    """
    job = Gaia.launch_job(query)
    result = job.get_results()

    if len(result) == 0:
        return None

    return int(result[0]['dr2_source_id'])


In [ ]:
def gaia_dr3_to_dr2(dr3_id):
    query = f"""
        SELECT TOP 1 dr2_source_id, angular_distance
        FROM gaiadr3.dr2_neighbourhood
        WHERE dr3_source_id = {dr3_id}
        ORDER BY angular_distance ASC
    """
    job = Gaia.launch_job(query)
    result = job.get_results()

    if len(result) == 0:
        return None

    return int(result[0]['dr2_source_id'])


In [ ]:
test_dr3 = df_mag['source_id'].iloc[0]
dr2_id = gaia_dr3_to_dr2(test_dr3)
dr2_id



In [ ]:
ddef gaia_dr2_to_tic(dr2_id):
    result = Catalogs.query_criteria(
        catalog="TIC",
        GAIAID=dr2_id   # CORRECT TIC column for Gaia DR2
    )

    if len(result) == 0:
        return None

    return int(result[0]["ID"])



In [ ]:
dr3 = df_mag['source_id'].iloc[1]
dr2 = gaia_dr3_to_dr2(dr3)
tic = gaia_dr2_to_tic(dr2)

dr3, dr2, tic


In [ ]:
def gaia_dr3_to_tic(dr3_id):
    dr2_id = gaia_dr3_to_dr2(dr3_id)
    if dr2_id is None:
        return None
    return gaia_dr2_to_tic(dr2_id)


In [ ]:
gaia_dr3_to_tic(df_mag['source_id'].iloc[0])


In [ ]:
from lightkurve import search_lightcurvefile

def get_tess_lightcurve(tic_id):
    """
    Downloads TESS PDCSAP light curve for a given TIC ID.
    Returns a LightCurve object or None if unavailable.
    """
    search_result = search_lightcurvefile(f"TIC {tic_id}")

    if len(search_result) == 0:
        print("No TESS data found for TIC:", tic_id)
        return None

    lcf = search_result.download()
    lc = lcf.PDCSAP_FLUX.remove_nans()
    return lc


In [ ]:
lc = get_tess_lightcurve(318754598)
lc


In [ ]:
lc.plot()
plt.title("Raw TESS Light Curve (TIC 318754598)")
plt.show()


In [ ]:
lc_clean = lc.remove_outliers().flatten()
lc_clean.plot()
plt.title("Detrended TESS Light Curve")
plt.show()


In [ ]:
from astropy.timeseries import LombScargle
import numpy as np

frequency, power = LombScargle(
    lc_clean.time.value,
    lc_clean.flux.value
).autopower()

period = 1 / frequency[np.argmax(power)]
period


In [ ]:
plt.plot(1/frequency, power)
plt.xlabel("Period (days)")
plt.ylabel("Power")
plt.title("Periodogram — Lomb Scargle")
plt.show()


In [ ]:
lc_folded = lc_clean.fold(period)
lc_folded.plot()
plt.title(f"Phase-Folded Light Curve (P = {period:.5f} days)")
plt.show()


In [ ]:
lc.plot().figure.savefig("raw_lc.png")
lc_clean.plot().figure.savefig("clean_lc.png")

plt.plot(1/frequency, power)
plt.xlabel("Period"); plt.ylabel("Power")
plt.title("Periodogram")
plt.savefig("periodogram.png")

lc_folded.plot().figure.savefig("phase_folded.png")


In [ ]:
from astropy.timeseries import LombScargle
import numpy as np

# Restrict to realistic stellar frequencies
frequency, power = LombScargle(
    lc_clean.time.value,
    lc_clean.flux.value
).autopower(
    minimum_frequency = 1/100,   # max period = 100 days
    maximum_frequency = 1/0.02   # min period = 0.02 days (30 minutes)
)

period = 1 / frequency[np.argmax(power)]
period


In [ ]:
plt.plot(1/frequency, power)
plt.xlabel("Period (days)")
plt.ylabel("Power")
plt.title("Periodogram (Filtered Frequency Range)")
plt.show()


In [ ]:
period


In [ ]:
lc_folded = lc_clean.fold(period)
lc_folded.plot()
plt.title(f"Phase-Folded TESS Light Curve (P = {period:.5f} days)")
plt.show()


In [ ]:
def color_to_temperature(bp_rp):
    """
    Approximate BP-RP to temperature relation.
    Valid for main-sequence stars approx 4000–9000K.
    """
    # Polynomial fit from Pecaut & Mamajek (2013)
    return 8500 - 3500 * (bp_rp - 0.5)

df_mag["Teff"] = df_mag["bp_rp"].apply(color_to_temperature)
df_mag["Teff"].head()


In [ ]:
df_mag = df_mag[df_mag["parallax"] > 0]
df_mag["distance_pc"] = 1000 / df_mag["parallax"]
df_mag["M_G"] = df_mag["phot_g_mean_mag"] - 5 * np.log10(df_mag["distance_pc"] / 10)


In [ ]:
def luminosity_from_MG(M_G):
    M_sun = 4.67  # Gaia G-band absolute magnitude of the Sun
    return 10 ** ((M_sun - M_G) / 2.5)

df_mag["Luminosity"] = df_mag["M_G"].apply(luminosity_from_MG)


In [ ]:
df_mag["Radius_Rsun"] = (
    (df_mag["Luminosity"]) ** 0.5 *
    (5772 / df_mag["Teff"]) ** 2
)


In [ ]:
df_mag["Mass_Msun"] = df_mag["Luminosity"] ** (1/3.5)


In [ ]:
df_mag["Age_Gyr"] = 10 * (df_mag["Mass_Msun"] ** -2.5)


In [ ]:
params = df_mag.loc[df_mag['source_id'] == df_mag['source_id'].iloc[0],
                    ["Teff", "Luminosity", "Radius_Rsun", "Mass_Msun", "Age_Gyr"]]
params
